In [32]:
import pandas as pd
from difflib import SequenceMatcher

# Read the data into dataframes
M1 = pd.read_csv("./data/MCPMaser2023_rudmingb.csv")
M2 = pd.read_csv("./data/masers_with_lum_2018.csv")

# Define the similarity calculation function
def calculate_similarity(string1, string2):
    matcher = SequenceMatcher(None, string1, string2)
    similarity = matcher.ratio() * 100
    return similarity

matched_count = 0
unmatched_count = 0
# Iterate over each entry in M1.Source_Name column
for source_name in M1['Source_Name']:
    max_similarity = 0
    matching_entry = None
    passes_test = False
    final_min_sim = 0
    final_avg_len = 0
    
    # Compare the source_name with each entry in M2.Galaxy column
    for galaxy in M2['Galaxy']:
        # Calculate the minimum similarity threshold based on string lengths
        lengthy = (len(source_name) + len(galaxy))/2   
        min_similarity_threshold = (66/lengthy)+57  # Adjust the multiplier as needed: https://www.desmos.com/calculator/aj8aa5qwar
        similarity = calculate_similarity(source_name, galaxy)
        if similarity > max_similarity:
            max_similarity = similarity
            matching_entry = galaxy
            final_min_sim = min_similarity_threshold
            final_avg_len = lengthy
            passes_test = False
            if similarity >= min_similarity_threshold:
                passes_test = True


    # Check if a matching entry was found
    if passes_test:
        # print(f"M1.Source_Name: {source_name}")
        # print(f"Matching entry in M2.Galaxy: {matching_entry}")
        # print(f"Highest similarity: {max_similarity}")
        # print(f"With Min similarity: {final_min_sim}")
        # print(f"And avg len: {final_avg_len}")
        # print("----------------------------------")
        matched_count +=1
    else:
        print(f"No match found for M1.Source_Name: {source_name}")
        print(f"The matching entry in M2.Galaxy would have been: {matching_entry}")
        print(f"With similarity: {max_similarity}")
        print(f"With Min similarity: {final_min_sim}")
        print(f"And avg len: {final_avg_len}")

        print("----------------------------------")
        unmatched_count +=1
print(matched_count)
print(unmatched_count)

----------------------------------
----------------------------------
----------------------------------
----------------------------------
----------------------------------
No match found for M1.Source_Name: M31
The matching entry in M2.Galaxy would have been: M33
With similarity: 66.66666666666666
With Min similarity: 79.0
And avg len: 3.0
----------------------------------
----------------------------------
----------------------------------
----------------------------------
No match found for M1.Source_Name: SMC
The matching entry in M2.Galaxy would have been: M33
With similarity: 33.33333333333333
With Min similarity: 79.0
And avg len: 3.0
----------------------------------
----------------------------------
----------------------------------
----------------------------------
----------------------------------
----------------------------------
----------------------------------
----------------------------------
----------------------------------
------------------------------